# 🚀 Beginner Projects & Exercises

**Phase 1: Beginner - Practical Application**

**Prerequisites**: Basic Scala syntax and collections

**Build confidence with real projects that combine all beginner concepts!**

---

## 🧮 Calculator Application

Build a command-line calculator that demonstrates all beginner concepts

In [ ]:
// Calculator Data Structures and Types
sealed trait Operation
case object Add extends Operation
case object Subtract extends Operation
case object Multiply extends Operation
case object Divide extends Operation

case class CalculatorError(message: String)
case class OperationResult(value: Double, operation: String)

object CalculatorApp {

  // Core calculator logic
  def calculate(op: Operation, a: Double, b: Double): Either[CalculatorError, OperationResult] = {
    val result = op match {
      case Add      => a + b
      case Subtract => a - b
      case Multiply => a * b
      case Divide   =>
        if (b == 0) return Left(CalculatorError("Division by zero"))
        a / b
    }
    
    val opName = op match {
      case Add      => "addition"
      case Subtract => "subtraction" 
      case Multiply => "multiplication"
      case Divide   => "division"
    }
    
    Right(OperationResult(result, opName))
  }

  // Parse operation from string
  def parseOperation(str: String): Option[Operation] = str.toLowerCase match {
    case "+" | "add"      => Some(Add)
    case "-" | "subtract" => Some(Subtract)
    case "*" | "multiply" => Some(Multiply)
    case "/" | "divide"   => Some(Divide)
    case _                 => None
  }

  // Interactive calculator session
  def runCalculator(): Unit = {
    println("=== Scala Calculator ===")
    println("Enter calculations like: 5 + 3 or 'quit' to exit")
    println("Supported operations: +, -, *, /")

    var continue = true
    var history = List[OperationResult]()
    
    while (continue) {
      print("\n> ")
      val input = scala.io.StdIn.readLine().trim
      
      input match {
        case "quit" | "exit" =>
          continue = false
          println("Goodbye!")
          
        case "history" =>
          println("\n=== Calculation History ===")
          if (history.isEmpty) {
            println("No calculations yet")
          } else {
            history.zipWithIndex.foreach { case (result, index) =>
              println(f"${index + 1}%d. ${result.operation} = ${result.value}%.2f")
            }
          }
          
        case "clear" =>
          history = Nil
          println("History cleared")
          
        case expr if expr.contains(" ") =>
          val parts = expr.split("\\s+")
          if (parts.length == 3) {
            (parseDouble(parts(0)), parseOperation(parts(1)), parseDouble(parts(2))) match {
              case (Some(a), Some(op), Some(b)) =>
                calculate(op, a, b) match {
                  case Right(result) =>
                    println(f"$a%.2f ${parts(1)} $b%.2f = ${result.value}%.2f")
                    history = result :: history
                  case Left(error) =>
                    println(s"Error: ${error.message}")
                }
              case _ =>
                println("Invalid input. Use format: number operation number")
                println("Example: 10 + 5")
            }
          } else {
            println("Invalid format. Use: number operation number")
          }
          
        case _ =>
          println("Commands: expression (e.g. '5 + 3'), history, clear, quit")
      }
    }
  }
  
  private def parseDouble(s: String): Option[Double] = {
    try {
      Some(s.toDouble)
    } catch {
      case _: NumberFormatException => None
    }
  }
}

// Test calculator functionality
println("Calculator Application Ready")
println("To run interactive calculator: CalculatorApp.runCalculator()")
println()

// Test individual calculations
val calc = CalculatorApp
println("=== Calculator Tests ===")

def testCalc(op: Operation, a: Double, b: Double): Unit = {
  calc.calculate(op, a, b) match {
    case Right(result) =>
      val opSymbol = op match {
        case Add => "+"
        case Subtract => "-"
        case Multiply => "*"
        case Divide => "/"
      }
      println(f"$a%.1f $opSymbol $b%.1f = ${result.value}%.2f")
    case Left(error) => println(s"Error: ${error.message}")
  }
}

// Basic operations
testCalc(Add, 10, 5)
testCalc(Subtract, 10, 5)
testCalc(Multiply, 10, 5)
testCalc(Divide, 10, 5)
// Division by zero test
testCalc(Divide, 10, 0)

// Operation parsing
println(s"\nParsed '+': ${calc.parseOperation("+")}")
println(s"Parsed 'add': ${calc.parseOperation("add")}")
println(s"Parsed 'invalid': ${calc.parseOperation("invalid")}")

println("\nCalculator tests completed!")
println()

## 📊 Simple Inventory System

Build a basic inventory management system using case classes and collections

In [ ]:
// Inventory System Data Models
case class Product(id: String, name: String, price: Double, category: String)
case class InventoryItem(product: Product, quantity: Int, location: String)
case class InventoryReport(
  totalItems: Int,
  totalValue: Double,
  categories: Map[String, Int],
  lowStockItems: List[InventoryItem]
)

object InventoryManagement {
  private var inventory = Set[InventoryItem]()
  private val lowStockThreshold = 10

  // Add new inventory item
  def addItem(item: InventoryItem): Either[String, Unit] = {
    // Check if product already exists
    val existingProduct = inventory.find(_.product.id == item.product.id)
    existingProduct match {
      case Some(existing) =>
        Left(s"Product ${item.product.id} already exists at location ${existing.location}")
      case None =>
        inventory = inventory + item
        Right(())
    }
  }

  // Update quantity
  def updateQuantity(productId: String, newQuantity: Int): Either[String, Unit] = {
    if (newQuantity < 0) return Left("Quantity cannot be negative")
    
    val itemOpt = inventory.find(_.product.id == productId)
    itemOpt match {
      case Some(item) =>
        val updated = item.copy(quantity = newQuantity)
        inventory = inventory - item + updated
        Right(())
      case None =>
        Left(s"Product $productId not found")
    }
  }

  // Generate inventory report
  def generateReport(): InventoryReport = {
    val totalItems = inventory.map(_.quantity).sum
    val totalValue = inventory.map(item => item.product.price * item.quantity).sum
    
    val categories = inventory.groupBy(_.product.category)
      .map { case (category, items) =>
        category -> items.map(_.quantity).sum
      }
    
    val lowStockItems = inventory.filter(_.quantity <= lowStockThreshold).toList
    
    InventoryReport(totalItems, totalValue, categories, lowStockItems)
  }

  // Search functionality
  def searchByCategory(category: String): List[InventoryItem] = {
    inventory.filter(_.product.category.toLowerCase.contains(category.toLowerCase)).toList
  }

  def searchByName(name: String): List[InventoryItem] = {
    inventory.filter(_.product.name.toLowerCase.contains(name.toLowerCase)).toList
  }

  // Price calculations
  def getTotalValue(): Double = {
    inventory.map(item => item.product.price * item.quantity).sum
  }

  def getAveragePrice(): Double = {
    val items = inventory.filter(_.quantity > 0)
    if (items.isEmpty) 0.0
    else items.map(_.product.price).sum / items.size
  }

  // Utility methods
  def listAllItems(): List[InventoryItem] = inventory.toList

  def clearInventory(): Unit = {
    inventory = Set.empty[InventoryItem]
  }

  def getItem(productId: String): Option[InventoryItem] = {
    inventory.find(_.product.id == productId)
  }
}

// Test the inventory system
println("=== Inventory Management System ===")

// Create sample products
val laptop = Product("P001", "Gaming Laptop", 1200.00, "Electronics")
val mouse = Product("P002", "Wireless Mouse", 25.50, "Electronics")
val desk = Product("P003", "Standing Desk", 350.00, "Furniture")
val chair = Product("P004", "Office Chair", 200.00, "Furniture")

// Add items to inventory
val inventory = InventoryManagement

// Add items
def testAddItem(item: InventoryItem): Unit = {
  inventory.addItem(item) match {
    case Right(_) => println(s"✓ Added ${item.product.name}")
    case Left(error) => println(s"✗ Error: $error")
  }
}

testAddItem(InventoryItem(laptop, 5, "Warehouse A"))
testAddItem(InventoryItem(mouse, 25, "Warehouse A"))
testAddItem(InventoryItem(desk, 3, "Warehouse B"))
testAddItem(InventoryItem(chair, 8, "Warehouse B"))  // Low stock

// Try to add duplicate
testAddItem(InventoryItem(laptop, 10, "Warehouse C"))

// Update quantities
println("\n=== Quantity Updates ===")
inventory.updateQuantity("P004", 15)  // Restock chairs
inventory.updateQuantity("P002", 8)   // Low stock mouse
inventory.updateQuantity("P999", 5)   // Non-existent product

// Generate report
println("\n=== Inventory Report ===")
val report = inventory.generateReport()

println(s"Total items in stock: ${report.totalItems}")
println(f"Total inventory value: $$${report.totalValue}%.2f")
println(f"Average price: $$${inventory.getAveragePrice()}%.2f")
println(s"\nCategory breakdown:")
report.categories.foreach { case (category, count) =>
  println(f"  $category%12s: $count%3d items")
}

println(s"\nLow stock items (${report.lowStockItems.size}):")
report.lowStockItems.foreach { item =>
  println(f"  ${item.product.name}%-15s: ${item.quantity} in stock")
}

// Search functionality
println("\n=== Search Results ===")
println("Items in Electronics:")
inventory.searchByCategory("Electronics").foreach { item =>
  println(f"  ${item.product.name}%-15s: ${item.quantity}")
}

println("\nSearch for 'Desk':")
inventory.searchByName("Desk").foreach { item =>
  println(f"  ${item.product.name}%-15s: ${item.quantity}")
}

println("\nInventory system demonstration completed!")
println()

## 🎯 Student Grade Management System

Comprehensive student management with grades, averages, and performance analysis

In [ ]:
// Student Grade Management Data Models
case class Student(id: String, name: String, gradeLevel: Int)
case class Subject(name: String, credits: Int, passingGrade: Double = 60.0)

case class Grade(
  studentId: String,
  subject: Subject,
  score: Double,
  semester: String,
  dateAchieved: String
)

case class GradeAnalysis(
  student: Student,
  grades: List[Grade],
  gpa: Double,
  totalCredits: Int,
  passedSubjects: Int,
  failedSubjects: Int,
  performanceLevel: String
)

case class ClassStatistics(
  subject: Subject,
  totalStudents: Int,
  averageGrade: Double,
  passingRate: Double,
  highestGrade: Double,
  lowestGrade: Double,
  gradeDistribution: Map[String, Int]  // A, B, C, D, F
)

object GradeManagement {
  private var students = Map[String, Student]()
  private var grades = List[Grade]()
  private var subjects = Map[String, Subject]()

  // Student management
  def addStudent(student: Student): Either[String, Unit] = {
    if (students.contains(student.id)) {
      Left(s"Student ${student.id} already exists")
    } else {
      students = students + (student.id -> student)
      Right(())
    }
  }

  // Subject management
  def addSubject(subject: Subject): Either[String, Unit] = {
    val subjectKey = subject.name.toLowerCase.replace(" ", "")
    if (subjects.contains(subjectKey)) {
      Left(s"Subject '${subject.name}' already exists")
    } else {
      subjects = subjects + (subjectKey -> subject)
      Right(())
    }
  }

  // Grade recording
  def recordGrade(grade: Grade): Either[String, Unit] = {
    if (!students.contains(grade.studentId)) {
      return Left(s"Student ${grade.studentId} not found")
    }
    
    val subjectKey = grade.subject.name.toLowerCase.replace(" ", "")
    if (!subjects.contains(subjectKey)) {
      return Left(s"Subject '${grade.subject.name}' not found")
    }
    
    if (grade.score < 0 || grade.score > 100) {
      return Left("Grade must be between 0 and 100")
    }
    
    grades = grades :+ grade
    Right(())
  }

  // GPA calculation
  def calculateGPA(gradeValue: Double): Double = {
    if (gradeValue >= 90) 4.0
    else if (gradeValue >= 80) 3.0
    else if (gradeValue >= 70) 2.0
    else if (gradeValue >= 60) 1.0
    else 0.0
  }

  // Student performance analysis
  def analyzeStudent(studentId: String): Either[String, GradeAnalysis] = {
    students.get(studentId) match {
      case None => Left(s"Student $studentId not found")
      case Some(student) =>
        val studentGrades = grades.filter(_.studentId == studentId)
        
        if (studentGrades.isEmpty) {
          return Left(s"No grades found for student $studentId")
        }

        val totalCredits = studentGrades.map(_.subject.credits).sum
        val weightedGrades = studentGrades.map { grade =>
          val gpaPoints = calculateGPA(grade.score)
          (gpaPoints * grade.subject.credits, grade.subject.credits)
        }
        
        val totalWeightedPoints = weightedGrades.map(_._1).sum
        val totalGpaCredits = weightedGrades.map(_._2).sum
        val gpa = if (totalGpaCredits > 0) totalWeightedPoints / totalGpaCredits else 0.0
        
        val passedSubjects = studentGrades.count(_.score >= _.subject.passingGrade)
        val failedSubjects = studentGrades.length - passedSubjects
        
        val performanceLevel = 
          if (gpa >= 3.5) "Excellent"
          else if (gpa >= 3.0) "Good"
          else if (gpa >= 2.0) "Satisfactory"
          else "Needs Improvement"
        
        Right(GradeAnalysis(
          student,
          studentGrades,
          gpa,
          totalCredits,
          passedSubjects,
          failedSubjects,
          performanceLevel
        ))
    }
  }

  // Class statistics
  def analyzeClass(subjectName: String): Either[String, ClassStatistics] = {
    val subjectKey = subjectName.toLowerCase.replace(" ", "")
    subjects.get(subjectKey) match {
      case None => Left(s"Subject '$subjectName' not found")
      case Some(subject) =>
        val subjectGrades = grades.filter(_.subject.name == subjectName)
        
        if (subjectGrades.isEmpty) {
          return Left(s"No grades found for subject '$subjectName'")
        }
        
        val scores = subjectGrades.map(_.score)
        val totalStudents = subjectGrades.length
        val averageGrade = scores.sum / totalStudents
        val passingRate = subjectGrades.count(_.score >= subject.passingGrade).toDouble / totalStudents * 100
        val highestGrade = scores.max
        val lowestGrade = scores.min
        
        val gradeDistribution = scores.groupBy { score =>
          if (score >= 90) "A"
          else if (score >= 80) "B"
          else if (score >= 70) "C"
          else if (score >= 60) "D"
          else "F"
        }.mapValues(_.length)
        
        Right(ClassStatistics(
          subject,
          totalStudents,
          averageGrade,
          passingRate,
          highestGrade,
          lowestGrade,
          gradeDistribution
        ))
    }
  }

  // Utility functions
  def getAllStudents(): List[Student] = students.values.toList
  def getAllSubjects(): List[Subject] = subjects.values.toList
  def getAllGrades(): List[Grade] = grades
  
  def clearData(): Unit = {
    students = Map.empty
    grades = List.empty
    subjects = Map.empty
  }
}

// Test the grade management system
println("=== Student Grade Management System ===")

val gradeSystem = GradeManagement

// Setup subjects
val math Subject = Subject("Mathematics", 4)
val english Subject = Subject("English", 3)
val science = Subject("Science", 4)
val history = Subject("History", 3)

gradeSystem.addSubject(math)}
gradeSystem.addSubject(english)}
gradeSystem.addSubject(science)
gradeSystem.addSubject(history)

// Add students
val alice = Student("S001", "Alice Johnson", 10)
val bob = Student("S002", "Bob Smith", 10)
val charlie = Student("S003", "Charlie Wilson", 10)

gradeSystem.addStudent(alice)
gradeSystem.addStudent(bob)
gradeSystem.addStudent(charlie)

// Record grades
// Alice's grades
gradeSystem.recordGrade(Grade("S001", math, 95.0, "Fall 2023", "2023-12-15"))
gradeSystem.recordGrade(Grade("S001", english, 88.0, "Fall 2023", "2023-12-16"))
gradeSystem.recordGrade(Grade("S001", science, 92.0, "Fall 2023", "2023-12-17"))
gradeSystem.recordGrade(Grade("S001", history, 85.0, "Fall 2023", "2023-12-18"))

// Bob's grades
gradeSystem.recordGrade(Grade("S002", math, 78.0, "Fall 2023", "2023-12-15"))
gradeSystem.recordGrade(Grade("S002", english, 82.0, "Fall 2023", "2023-12-16"))
gradeSystem.recordGrade(Grade("S002", science, 75.0, "Fall 2023", "2023-12-17"))
gradeSystem.recordGrade(Grade("S002", history, 90.0, "Fall 2023", "2023-12-18"))

// Charlie's grades
gradeSystem.recordGrade(Grade("S003", math, 88.0, "Fall 2023", "2023-12-15"))
gradeSystem.recordGrade(Grade("S003", english, 91.0, "Fall 2023", "2023-12-16"))
gradeSystem.recordGrade(Grade("S003", science, 87.0, "Fall 2023", "2023-12-17"))
gradeSystem.recordGrade(Grade("S003", history, 83.0, "Fall 2023", "2023-12-18"))

// Analyze student performance
println("=== Student Performance Analysis ===")

def printStudentAnalysis(studentId: String): Unit = {
  gradeSystem.analyzeStudent(studentId) match {
    case Right(analysis) =>
      println(s"\nStudent: ${analysis.student.name} (Grade: ${analysis.student.gradeLevel})")
      println(f"GPA: ${analysis.gpa}%.2f")
      println(s"Total Credits: ${analysis.totalCredits}")
      println(s"Passed Subjects: ${analysis.passedSubjects}")
      println(s"Failed Subjects: ${analysis.failedSubjects}")
      println(s"Performance Level: ${analysis.performanceLevel}")
      
      println("Grades:")
      analysis.grades.foreach { grade =>
        val status = if (grade.score >= grade.subject.passingGrade) "PASS" else "FAIL"
        println(f"  ${grade.subject.name}%-12s: ${grade.score}%5.1f ($status)")
      }
      
    case Left(error) =>
      println(s"Error analyzing student $studentId: $error")
  }
}

printStudentAnalysis("S001")  // Alice
printStudentAnalysis("S002")  // Bob
printStudentAnalysis("S003")  // Charlie

// Class statistics
println("\n=== Class Statistics ===")

def printClassStats(subjectName: String): Unit = {
  gradeSystem.analyzeClass(subjectName) match {
    case Right(stats) =>
      println(s"\nSubject: ${stats.subject.name} (${stats.subject.credits} credits)")
      println(s"Total Students: ${stats.totalStudents}")
      println(f"Average Grade: ${stats.averageGrade}%.1f")
      println(f"Passing Rate: ${stats.passingRate}%.1f%")
      println(f"Grade Range: ${stats.lowestGrade}%.1f - ${stats.highestGrade}%.1f")
      
      println("Grade Distribution:")
      "ABCDEF".foreach { letter =>
        val count = stats.gradeDistribution.getOrElse(letter.toString, 0)
        val percent = if (stats.totalStudents > 0) count.toDouble / stats.totalStudents * 100 else 0.0
        println(f"  Grade $letter: $count students (${percent}%.1f%)")
      }
      
    case Left(error) =>
      println(s"Error analyzing class $subjectName: $error")
  }
}

printClassStats("Mathematics")
printClassStats("English")

println("\nGrade management system demonstration completed!")
println()
println("--- Summary ---")
println("Built complete grade management system with:")
println("✓ Student and subject management")
println("✓ Grade recording and validation") 
println("✓ GPA calculation and performance analysis")
println("✓ Class statistics and grade distributions")
println("✓ All using Scala case classes and collections")
println()

## 📋 Beginner Project Checklist

### **Calculator Application** ✅
- [x] Defined custom data types (Operation, CalculatorError, OperationResult)
- [x] Implemented sealed traits for operation types
- [x] Used pattern matching for calculations
- [x] Added error handling with Either
- [x] Implemented parsing and input validation
- [x] Added interactive command-line interface
- [x] Included calculation history and command parsing

### **Inventory Management System** ✅
- [x] Used case classes for data modeling
- [x] Implemented immutable Set for inventory storage
- [x] Added validation logic with Either types
- [x] Implemented search and filter operations
- [x] Added calculation methods (total value, averages)
- [x] Created comprehensive reporting functionality
- [x] Used collections operations (filter, groupBy, map)

### **Grade Management System** ✅
- [x] Built complex domain model with relationships
- [x] Implemented data validation and business rules
- [x] Added comprehensive analysis and reporting
- [x] Used Maps and Lists for data storage
- [x] Implemented GPA calculation algorithms
- [x] Added grade distribution and statistics
- [x] Used pattern matching and functional operations

---

### **Scala Beginner Skills Demonstrated**
1. **Data Types & Classes**: Case classes, sealed traits, custom types
2. **Collections**: Lists, Sets, Maps, operations like map/filter/groupBy
3. **Pattern Matching**: Exhaustive matching, case expressions
4. **Error Handling**: Either, Option, custom error types
5. **Functions**: Higher-order functions, function composition
6. **Immutability**: Pure functions, immutable data structures
7. **Control Flow**: If-else, pattern matching, for-comprehensions
8. **Input/Output**: Console I/O, string processing, parsing
9. **Validation**: Input validation, business rule enforcement
10. **Data Transformation**: Map/reduce patterns, data pipelines

### **Next Steps**
**🎯 Ready for Phase 2!** These projects demonstrate all beginner concepts:
- Move to **Intermediate Phase** (Functional Programming, Futures, Testing)
- All beginner foundations are solid
- Ready to build more complex systems